# Handling Categorical Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv('home-data-for-ml-course/train.csv', index_col='Id')
X_test = pd.read_csv('home-data-for-ml-course/test.csv', index_col='Id')

In [3]:
# Remove data with blank SalePrice
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X['SalePrice']
X.drop(['SalePrice'], axis=1, inplace=True)

# Drop missing values
cols_with_missing = [col for col in X.columns
                    if X[col].isnull().any()]
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [5]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [7]:
# Option 1: Drop object types
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

17837.82570776256


In [8]:
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


In [9]:
# Option 2: Use a Label Encoder
from sklearn.preprocessing import LabelEncoder

object_cols = X_train.select_dtypes(['object'])

# Get cols with same values between training and validation
good_cat_cols = [col for col in object_cols
                if set(X_train[col]) == set(X_valid[col])]

bad_cat_cols = list(set(object_cols) - set(good_cat_cols))

# Drop the bad categorical columns
label_X_train = X_train.drop(bad_cat_cols, axis=1)
label_X_valid = X_valid.drop(bad_cat_cols, axis=1)

label_encoder = LabelEncoder()

for col in set(good_cat_cols):
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])
    
# Evaluate
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

17575.291883561644


In [10]:
# Cardinality check
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [11]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

# Separate low-cardinality columns (less than 10 unique values)
low_cardinality_cols = [col for col in object_cols
                       if X_train[col].nunique() < 10]

high_cardinality_cols = list(set(object_cols) - set(low_cardinality_cols))

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_train = pd.DataFrame(one_hot_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(one_hot_encoder.transform(X_valid[low_cardinality_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove all object columns from the training and validation data
num_only_X_train = X_train.drop(object_cols, axis=1)
num_only_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_only_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_only_X_valid, OH_cols_valid], axis=1)

# Evaluate
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

17525.345719178084


In [14]:
# Decision: Do One Hot Encoding
final_OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_train = pd.DataFrame(final_OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(final_OH_encoder.transform(X_valid[low_cardinality_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_only_X_train = X_train.drop(object_cols, axis=1)
num_only_X_valid = X_valid.drop(object_cols, axis=1)

final_OH_X_train = pd.concat([num_only_X_train, OH_cols_train], axis=1)
final_OH_X_valid = pd.concat([num_only_X_valid, OH_cols_valid], axis=1)

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_OH_X_train, y_train)

validation_predictions = model.predict(final_OH_X_valid)
print(mean_absolute_error(y_valid, validation_predictions))

17525.345719178084


In [25]:
# Remove nulls from test data
X_test.dropna(axis=0, inplace=True)

# Apply the OH encoder on the test data
OH_cols_test = pd.DataFrame(final_OH_encoder.transform(X_test[low_cardinality_cols]))
OH_cols_test.index = X_test.index
num_only_X_test = X_test.drop(object_cols, axis=1)
final_OH_X_test = pd.concat([num_only_X_test, OH_cols_test], axis=1)

test_predictions = model.predict(final_OH_X_test)
test_predictions
# Drop the object columns from test data
# Concatenate the OH Encoding result columns to test data
# Predict()
# Output file

array([127654.5 , 157672.  , 182222.9 , ..., 153695.25, 109710.85,
       223045.55])

In [26]:
# Output predictions
output = pd.DataFrame({'Id': final_OH_X_test.index,
                     'SalePrice': test_predictions})
output.to_csv('submission2.csv', index=False)